<a href="https://colab.research.google.com/github/restrepo/OpenAlex/blob/main/json_total.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAlex works for Colombia
## Concepts associated to "Physics" in UdeA

In [1]:
import json
import pandas as pd
from google.colab import files
#From https://gist.github.com/korakot/072f1ea70d3c3267ff2c64b61d78c89c
import json
from IPython.display import HTML

def JSON(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)

### Load the json file of works for Colombia

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/json/openalexco.json','r') as miarchivo:
    objeto=miarchivo.read()
objeto=json.loads(objeto)

In [ ]:
len(objeto)

343057

Filter works with ror:

https://ror.org/03bp5hc83

In [ ]:
objeto=[ c for c in objeto if any(
    [ any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')]) 
    for a in c.get('authorships')])
]
len(objeto)

34291

Define the filter which returns `True or False`

In [ ]:
def filter_ror(authorships,ror="https://ror.org/03bp5hc83"):
  return any([any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')])  for a in authorships])

In [ ]:
len([ c for c in objeto if filter_ror(c.get('authorships'))])

34291

Filter with Pandas

In [ ]:
objeto=pd.DataFrame(objeto)

In [ ]:
objeto[objeto['authorships'].apply(filter_ror)].shape

(34291, 25)

### Explore concept levels from Excel file

In [2]:
cp=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vT9JrvUwanM-z-hCZ1SSaxRx2wfzLV3RVvm5BJaf8TABBmpIwKt7iYt1sVLbImVevh9YNaZK9Y7sfI9/pub?output=xlsx')

In [ ]:
cp[cp['display_name'].str.lower().str.contains('complex')][:3]#['display_name']#.to_list()

,openalex_id,display_name,normalized_name,level,wikidata_id,parent_display_names,parent_ids
377,https://openalex.org/c34947359,Complex network,complex network,2.0,https://www.wikidata.org/wiki/Q665189,"Combinatorics, World Wide Web","https://openalex.org/C114614502, https://opena..."
1241,https://openalex.org/c2778445095,Sample complexity,sample complexity,2.0,https://www.wikidata.org/wiki/Q18354077,NaN,NaN
2527,https://openalex.org/c47822265,Complex system,complex systems,2.0,https://www.wikidata.org/wiki/Q854457,"Artificial intelligence, Social science","https://openalex.org/C154945302, https://opena..."


#### Level 1: Physics concepts
The 'parent_display_names' column has some `NaN`, which is a `float`, and therefore must be converted to `str`, before the string operations to avoid errors. `Physics'` may appears along with other concepts in the `'parent_display_names'` and therefore the `str.contains` method of Pandas is the best option here:

In [3]:
ph1=cp[ (cp['level']==1) & (cp['parent_display_names'].astype(str).str.contains('Physics')) 
      ]['display_name'].to_list()

In [4]:
len(ph1)

27

#### Level 2: Physics concepts
the `str.split` method of Pandas, conver the comma-space sepparated list into a Python list. Remember that the `set` object of Python has the `intersection` method directly

In [5]:
ph2=cp[(cp['level']==2) & (cp['parent_display_names'].astype(str).str.split(', ').apply(
    lambda l: set(l).intersection(set(ph1))).apply(len)>0)]['display_name'].to_list()
len(ph2)

2979

### UdeA Physics articles

In [ ]:
concept='Physics'
level=0
possible_physics=objeto[objeto['concepts'].apply(lambda l: {concept}.intersection( 
    [d.get('display_name') for d in l if d.get('level')==level])).apply(len)>0]
possible_physics.shape


(1886, 25)

Level 1: UdeA Physics concepts

In [ ]:
ll=possible_physics['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==1]).intersection(ph1)).apply(list)
ph1_udea=set([item for subitem in ll for item in subitem])

Missing Level 1 concepts at UdeA

In [ ]:
set(ph1).difference(ph1_udea)

{'Geophysics', 'Medical physics'}

Level 2: UdeA Physics concepts

In [ ]:
ll=possible_physics['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==2]).intersection(ph2)).apply(list)
ph2_udea=set([item for subitem in ll for item in subitem])

In [ ]:
len(ph2_udea)

507

### UdeA Physics concepts tree

In [ ]:
d={'Physics': dict( [(c,[]) for c in ph1_udea])}
for c in ph2_udea:
    lph1=set(ph1_udea).intersection( cp[cp['display_name']==c]['parent_display_names'].str.split(', ').iloc[0])
    for cc in lph1:
        if cc in d['Physics'] and c not in d['Physics'][cc]:
            d['Physics'][cc].append(c)
del cp


In [ ]:
JSON(d)

### Generic filter

In [ ]:
if isinstance(objeto,list):
    objeto=pd.DataFrame(objeto)
concept='Complex systems'
level=2
def filter_concept(df,concept,level):
    return df[df['concepts'].apply(lambda l: {concept}.intersection( 
    [d.get('display_name') for d in l if d.get('level')==level])).apply(len)>0]

In [ ]:
filter_concept(objeto,concept,level)

,id,doi,title,display_name,publication_year,publication_date,ids,host_venue,type,open_access,...,concepts,mesh,alternate_host_venues,referenced_works,related_works,cited_by_api_url,counts_by_year,updated_date,created_date,abstract_inverted_index


In [ ]:

f=open('openalexua.json','w')
json.dump(objeto,f)
f.close()

In [ ]:

x=set()
type(x)

In [ ]:
f=open('openalexua.json','r')
df=json.load(f)
f.close()